Demo of custom functions for MPOD that can be called easily from this notebook: 
- Ramping based on fixed intervals 
- Ramping like a voltage divider

In [10]:
from Driver.MPODClass import MPOD#Basic functionalities for HV Control
from Driver.MPODCustomFunctions import CustomFx #Custom functions for advanced control
import time
Crate = MPOD(IP='169.254.107.70', MIBdir='/usr/share/snmp/mibs')# default settings
FX = CustomFx(Crate, active_modules = [0], channel_names = ['Drift','GEM Top+','GEM Top-','GEM Mid+','GEM Mid-','GEM Low+','GEM Low-','None'])
#These settings are not the default and need to be kept the same for these to work. Order of channels can be changed as long as text names are moved accordingly

#Basic setup validation: 
print('Crate power state: ', Crate.GetPowerCrate())
print('All connected channels: ', Crate.GetAllNames())
for idx, name in enumerate(FX.channel_names):
    print('Channel ',FX.active_channels[idx], ' is designated as ', name)


Crate power state:  1
All connected channels:  [0, 1, 2, 3, 4, 5, 6, 7, 700, 701, 702, 703, 704, 705, 706, 707, 800, 801, 802, 803, 804, 805, 806, 807]
Channel  0  is designated as  Drift
Channel  1  is designated as  GEM Top+
Channel  2  is designated as  GEM Top-
Channel  3  is designated as  GEM Mid+
Channel  4  is designated as  GEM Mid-
Channel  5  is designated as  GEM Low+
Channel  6  is designated as  GEM Low-
Channel  7  is designated as  None


Now the ramping functions can be demonstrated.
1) Ramp all to zero
2) Ramp like a voltage divider (x2 absolute values, Drift & GEM, with ratios for dividing GEM voltage)
3) Ramp all by a fixed, global dV increment (x1 dV as input)
4) Ramp all with individual dV increments (x7 dV as input) NOT YET IMPLEMENTED
5) Ramp drift to a voltage, then set dV from drift NOT YET IMPLEMENTED


In [13]:
FX.RampAll()#default is to ramp all to zero
FX.Reset()

Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]


In [ ]:
#Voltage divider method: Requires user to review inputs (printed below) before sending by entering SEND
#User input box should appear in text search box at top of window. Enter SEND there! 
FX.RampAll()#Ramps all to zero before next set
time.sleep(2)

V_Drift = 20
V_GEM =  15 #Max V on GEMs
divider_fractions = [1,0.8,0.5,0.3,0.2,0.1]#top to bottom, fractions of V_GEM

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages())
print('V Actuals before ramp: ', Crate.GetAllVoltages())
FX.RampMethod('Voltage Divider',[20,15,*divider_fractions])
time.sleep(2)

print('V Targets after ramp: ',Crate.GetAllTargetVoltages())
print('V Actuals after ramp: ',Crate.GetAllVoltages())


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-9.778758049011, -7.610824584961, -6.244332790375, -3.832051992416, -2.081961154938, -1.229168653488, -2.143810987473, 0.146110400558, -0.082106798887, -0.063735410571, -0.081271752715, -0.03407202661, -0.048253431916, -0.044221319258, -0.056006200612, -0.042125351727, -0.006548100617, 0.023366380483, 0.010918006301, 0.011913204566, 0.017347643152, 0.022854302078, 0.019986711442, 0.000639718026]
Confirm settings below before sending
Drift voltage   on channel 0: 20 V
GEM Top voltage on channel 1 & 2: 15, 12.0 V
GEM Mid voltage on channel 3 & 4: 7.5, 4.5 V
GEM Low voltage on channel 5 & 6: 3.0, 1.5 V
V Targets after ramp:  [20.0, 15.0, 12.0, 7.5, 4.5, 3.0, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals

In [1]:
#Demo of ramping with aproximation of equal ramp rates: 
#TODO: Test this and make edits if needed. likely buggy... 
FX.RampTogether(channels = [0,1], target_voltage = [50,100])
# FX.RampTogether(FX.ActiveChannels,)
# RampTogether(self, channels = None, target_voltage = None

NameError: name 'FX' is not defined

In [ ]:
FX.RampAll()#Ramps all to zero before next set
time.sleep(2)

v_to_increment = 50

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages()[0:FX.n_channels])
print('V Actuals before ramp: ', Crate.GetAllVoltages()[0:FX.n_channels])
FX.RampMethod('Increment All', v_to_increment)#Increments all channels by 50 V
time.sleep(2)
print('V Targets after ramp: ',Crate.GetAllTargetVoltages()[0:FX.n_channels])
print('V Actuals after ramp: ',Crate.GetAllVoltages()[0:FX.n_channels])


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-0.950424790382, -1.198921322823, -0.879746675491, -0.59956663847, -0.998814225197, 0.15883782506, 0.095220267773, 0.143026918173]
V Targets after ramp:  [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 0.0]
V Actuals after ramp:  [-50.098361968994, -50.024505615234, -49.948768615723, -49.986663818359, -50.221656799316, -21.586284637451, -22.756118774414, 0.143026918173]


In [6]:
FX.RampAll()#Ramps all to zero before next set
time.sleep(2)
v_to_increment = [0,10,20,30,40,50,60]

print('V  Targets before ramp: ', Crate.GetAllTargetVoltages())
print('V Actuals before ramp: ', Crate.GetAllVoltages())
FX.RampMethod('Individual Increments', v_to_increment)#Increments each channels by increment value
time.sleep(2)
print('V Targets after ramp: ',Crate.GetAllTargetVoltages())
print('V Actuals after ramp: ',Crate.GetAllVoltages())


Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
V  Targets before ramp:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals before ramp:  [-0.951966524124, -2.439939498901, -6.355547428131, -9.906426429749, -12.981225967407, -0.3655205369, -0.483441233635, 0.146110400558, -0.084054663777, -0.074440971017, -0.086133122444, -0.039915800095, -0.054090894759, -0.042276605964, -0.056006200612, -0.047961674631, -0.006548100617, 0.021372124553, 0.01141603291, 0.01141446922, 0.016350800171, 0.023352328688, 0.016002878547, 0.000639718026]
V Targets after ramp:  [0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
V Actuals after ramp:  [-0.927297711372, -10.310474395752, -17.645290374756, -29.867343902588, -53.720802307129, -43.181144714355, -42.40340423584, 0.143026918173, -0.072367459536, -0.068601563573, -0.081271752715, -

In [24]:
Crate.SetPowerCrate(1)
x=Crate.Send('get','outputSwitch.u0 outputSwitch.u1')
print('1',x)
x=Crate.Send('set','outputSwitch.u0 i 0 outputSwitch.u1 i 0')
print('2',x)

x=Crate.Send('get','outputSwitch.u0 outputSwitch.u1')
print('3',x)

x=Crate.Send('get','moduleStatus.ma0 outputSwitch.u1')
print('4',x)
# Crate.SetPowerCrate(0)

1 WIENER-CRATE-MIB::outputSwitch.u0 = INTEGER: off(0)
WIENER-CRATE-MIB::outputSwitch.u1 = INTEGER: off(0)
2 WIENER-CRATE-MIB::outputSwitch.u0 = INTEGER: off(0)
WIENER-CRATE-MIB::outputSwitch.u1 = INTEGER: off(0)
3 WIENER-CRATE-MIB::outputSwitch.u0 = INTEGER: off(0)
WIENER-CRATE-MIB::outputSwitch.u1 = INTEGER: off(0)
4 WIENER-CRATE-MIB::moduleStatus.ma0 = BITS: 02 EE moduleIsInputError(6) moduleIsNoSumError(8) moduleIsNoRamp(9) moduleSafetyLoopIsGood(10) moduleIsGood(12) moduleSupplyIsGood(13) moduleTemperatureIsGood(14) 
WIENER-CRATE-MIB::outputSwitch.u1 = INTEGER: off(0)


In [ ]:
## Demo of sending multiple set (or get) commands simultaneously
FX.RampAll()
time.sleep(2)
print('Starting Voltages: ', Crate.GetAllVoltages())
print('Start logging commands for batch send')
Crate.SendMultiple('Start')

Crate.SetTargetVoltage(0,1)#cmd 1
Crate.SetTargetVoltage(1,5)#cmd 2
Crate.SetTargetVoltage(2,10)#cmd 3

print('Logged commands: ', Crate.gathered_commands)

Crate.SendMultiple('Pause')
print('Logging paused, allow command to pass thru until renabled')
print('All names command: ', Crate.GetAllNames())
print('Verify voltages are static: ', Crate.GetAllVoltages())
print('Now send batch of logged command')
Crate.SendMultiple('End')
time.sleep(2)
print('New voltages: ', Crate.GetAllVoltages())



Ramp values: [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
Starting Voltages:  [-0.877959728241, -0.996715426445, -2.782766342163, 0.064221650362, 0.099763989449, 0.160380005836, 0.096763372421, 0.143026918173, -0.083080738783, -0.052056610584, -0.083216302097, -0.038941830397, -0.05798253417, -0.052000157535, -0.056006200612, -0.045043513179, -0.006548100617, 0.021372124553, 0.009423945099, 0.010915733874, 0.017347643152, 0.023850355297, 0.019488729537, 0.000639718026]
Start logging commands for batch send
Logged commands:  ['sysMainSwitch.0 i 1', 'outputVoltage.u0 F 1', 'outputVoltage.u1 F 5', 'outputVoltage.u2 F 10']
Logging paused, allow command to pass thru until renabled
All names command:  [0, 1, 2, 3, 4, 5, 6, 7, 700, 701, 702, 703, 704, 705, 706, 707, 800, 801, 802, 803, 804, 805, 806, 807]
Verify voltages are static:  [-0.877959728241, -0.996715426445, -2.782766342163, 0.064221650362, 0.099763989449, 0.155753344297, 0.095220267773, 0.143026918173, -0.084054663777, -0.0